In [1]:
# laod the package
from google.colab import drive
drive.mount('/content/gdrive')
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os
from skimage import io, img_as_float
import numpy as np
from skimage.color import *
from skimage import feature, transform
from skimage.transform import resize
from skimage.feature import hog
from skimage.metrics import structural_similarity as ssim


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# define function that smooth the graph using hanning function
# https://numpy.org/doc/stable/reference/generated/numpy.hanning.html#:~:text=The%20Hanning%20window%20is%20a,an%20empty%20array%20is%20returned.
def smooth(x, window_len=10, window='hanning'): 
    s = np.r_[2 * x[0] - x[window_len:1:-1],
              x, 2 * x[-1] - x[-1:-window_len:-1]]

    w = getattr(np, window)(window_len)
    y = np.convolve(w / w.sum(), s, mode='same')
    return y[window_len - 1:-window_len + 1]

In [3]:
# Load the model
%%time
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 15.8 s, sys: 692 ms, total: 16.5 s
Wall time: 16.5 s


In [4]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3) #Decode a JPEG-encoded image to a uint8 tensor.
  return img

# showing cropped regions
# You can import any other required library
# showing cropped regions

# define a function 'hog_calculation(image_path)' which take the path of the image as input
# This function will calculate the HOG features for each cropped region detected in an image
# RETURN - hog_calculation() will return the minimum and/or maximum and/or average HOG features of the regions detected in an image
def hog_calculation(image_path):
  img = load_img(image_path)
  (frame_height, frame_width) = img.shape[:2]
  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  result = detector(converted_img)
  boxes = result["detection_boxes"] 
  all_hog_features = np.zeros([boxes.shape[0],288])
  for i in range(10):
    # print(np.squeeze(boxes)[i])
    ymin = int((np.squeeze(boxes)[i][0]*frame_height))
    xmin = int((np.squeeze(boxes)[i][1]*frame_width))
    ymax = int((np.squeeze(boxes)[i][2]*frame_height))
    xmax = int((np.squeeze(boxes)[i][3]*frame_width))
    cropped_img = img[ymin:ymax,xmin:xmax]
    scale_img = resize(cropped_img,(100,100))
    hog_feature,hog_image = hog(scale_img, orientations=8, pixels_per_cell=(16, 16),
                      cells_per_block=(1, 1), visualize=True, multichannel=True)
    all_hog_features[i,:] = hog_feature    
  image_hog_max = np.max(all_hog_features,axis=0)
  return image_hog_max




In [5]:
def find_factor(n):
    r = np.arange(1, int(n ** 0.5) + 1)
    x = r[np.mod(n, r) == 0]
    x_max = np.max(x)
    return (int(x_max), int(n/x_max))

In [6]:
def find_closest_factor(n):
  a, b = find_factor(n)
  temp = n
  y = 0
  step = 1
  while abs(a-b)>1:
    if y == 0:
      temp += step
      y += 1
    else:
      temp -= step
      y -= 1
    a, b = find_factor(temp)
    step += 1
  
  if a*b < n:
    change = 'reduce'
  elif a*b > n:
    change = 'enlarge'
  else:
    change = 'none'

  return a, b, change

In [7]:
# output overlapped image for images, 
# direction: horizontal/vertical
# overlap_weights: 0.x (the length of pixels overlapping)
# frame_size: because input image of this function might have different size, to fix this, we need the original frame size
def overlapping(img1, img2, direction, overlap_weight, slice, frame_size):
  h1, w1, c1 = img1.shape
  h2, w2, c2 = img2.shape
  height, width, channel = frame_size
  if direction == 'horizontal':
    overlap_width = int(width*overlap_weight)
    window = int(overlap_width/slice)
    boundary1 = np.arange(w1-slice*window,w1,window)
    boundary2 = np.arange(0,slice*window,window)
    shade1 = 1
    shade2 = 0
    for i in range(slice):
      img1[:,boundary1[i]:boundary1[i]+window,:] = img1[:,boundary1[i]:boundary1[i]+window,:]*shade1
      shade1 -= 1/(slice-1)
    for i in range(slice):
      img2[:,boundary2[i]:boundary2[i]+window,:] = img2[:,boundary2[i]:boundary2[i]+window,:]*shade2
      shade2 += 1/(slice-1)
    overlap_1 = img1[:,w1-window*slice:w1,:]
    overlap_2 = img2[:,0:window*slice,:]
    overlap_area = overlap_1 + overlap_2
    overlap_img = np.concatenate((img1[:,0:w1-window*slice,:],overlap_area,img2[:,window*slice:,:]),axis=1)
  else: # vertical
    if w1 != w2:
      img2 = (transform.resize(img2, (img1.shape))*255).astype('uint8')     
    overlap_height = int(height*overlap_weight)
    window = int(overlap_height/slice)
    boundary1 = np.arange(h1-slice*window,h1,window)
    boundary2 = np.arange(0,slice*window,window)
    shade1 = 1
    shade2 = 0
    for i in range(slice):
      img1[boundary1[i]:boundary1[i]+window,:,:] = img1[boundary1[i]:boundary1[i]+window,:,:]*shade1
      shade1 -= 1/(slice-1)
    for i in range(slice):
      img2[boundary2[i]:boundary2[i]+window,:,:] = img2[boundary2[i]:boundary2[i]+window,:,:]*shade2
      shade2 += 1/(slice-1)    
    overlap_1 = img1[h1-window*slice:h1,:,:]
    overlap_2 = img2[0:window*slice,:,:]
    overlap_area = overlap_1 + overlap_2
    overlap_img = np.concatenate((img1[0:h1-window*slice,:,:],overlap_area,img2[window*slice:,:,:]),axis=0)

  return overlap_img

In [8]:
def collage(img_list):
  num = len(img_list) # number of frames in the list
  x, y, change = find_closest_factor(num)
  if change == 'reduce':
    num_frame_change = num - x*y + y
    resize_width = int(img_list[0].shape[1] * y / num_frame_change)
    resize_height = int(resize_width/img_list[0].shape[1] * img_list[0].shape[0])
    for i in range(1,num_frame_change+1):
      img_list[-i] = (transform.resize(img_list[-i], (resize_height, resize_width, 3))*255).astype('uint8')
  elif change == 'enlarge': 
    num_frame_change = y - 1
    resize_width = int(img_list[0].shape[1] * y / num_frame_change)
    resize_height = int(resize_width/img_list[0].shape[1] * img_list[0].shape[0])
    for i in range(1,num_frame_change+1):
      img_list[-i] = (transform.resize(img_list[-i], (resize_height, resize_width, 3))*255).astype('uint8')
  else:
    pass


  if x == 1:
    output = img_list[0]
    for i in range(y-1):
      output = overlapping(output,img_list[i+1],'horizontal',0.4,33,img_list[0].shape)
    
  else:
    index = 0
    output_temp = []
    for i in range(x):
      output_h = img_list[index]
      if i == x-1:
        if change == 'enlarge':
          y = y-1
        elif change == 'reduce': 
          y = y+1
        else:
          pass
      for j in range(y-1):
        output_h = overlapping(output_h,img_list[j+1+index],'horizontal',0.4,33,img_list[0].shape)
      output_temp.append(output_h)
      index += y


    output_v = output_temp[0]
    for i in range(len(output_temp)-1):
      output_v = overlapping(output_v,output_temp[i+1],'vertical',0.4,33,output_temp[0].shape)
    output = output_v
    
  plt.imshow(output)
  plt.show()

In [9]:
def shading(img1, img2, coef):
    return coef*img1 + (1-coef)*img2

In [10]:
def gif_frame_making(frames,steps):
  ROOT_save = '/content/gdrive/My Drive/FIT5221_ass3_30399262_27462870/gif_frame_folder/'
  if 'gif_frame_folder' not in os.listdir('/content/gdrive/My Drive/FIT5221_ass3_30399262_27462870/'):
    os.mkdir(ROOT_save)
  else:
    pass
  num = len(frames)
  index = 0
  weight_list = np.linspace(1,0,steps)
  for i in range(num-1):
    weight = 1
    for j in weight_list:
      x = frames[i]
      y = frames[i+1]
      plt.imshow(shading(x,y,j).astype('uint8'))
      plt.show()
      io.imsave(ROOT_save+'f'+str(index)+'.jpeg',shading(x,y,j).astype('uint8'))
      index += 1

In [11]:
def generateCoolSummary(ROOT):
  
  list_files = os.listdir(ROOT)
  frames = []
  for i in range(len(list_files)):
    temp_frame = io.imread(ROOT+'t'+str(i)+'.jpeg')
    frames.append(temp_frame)

  height, width, channel = frames[0].shape
  dif_list = []
  for i in range(1,len(frames)):
    previous = rgb2gray(frames[i-1])
    current = rgb2gray(frames[i])
    dif = abs(previous - current)
    dif_sum = np.sum(dif)
    dif_mean = dif_sum/(height*width)
    dif_list.append(dif_mean)

  diff_array = np.array(dif_list)
  sm_diff_array = smooth(diff_array)
  frame_index = np.where((sm_diff_array[1:-1] > sm_diff_array[0:-2]) * (sm_diff_array[1:-1] > sm_diff_array[2:]))[0] + 1

  points = []
  for i in range(len(frame_index)):
    points.append(sm_diff_array[frame_index[i]])

  key_frames = []
  for i in frame_index:
    key_frames.append(frames[i])

  hog_frames = [] # hog of selected frames
  for i in frame_index:
    hog_frames.append(hog_calculation(ROOT+'t'+str(i)+'.jpeg'))

  hog_dist_dict = {}
  for i in range(len(hog_frames)):
    hog_dist_dict[i] = []
    for j in range(i+1,len(hog_frames)):
      hog_dist_dict[i].append(np.linalg.norm(hog_frames[i]-hog_frames[j]))
  hog_list = []
  for sublist in hog_dist_dict.values():
    for item in sublist:
      hog_list.append(item)

  delete_list_hog = set() # delete by hog distance <threshold
  threshold_hog = np.percentile(hog_list,10)
  for i in range(len(hog_dist_dict.keys())):
    for j in range(len(hog_dist_dict[i])):
      if hog_dist_dict[i][j]<=threshold_hog:
        delete_list_hog.add(i+1+j)
  remain = list(range(len(hog_frames)))
  for i in delete_list_hog:
    remain.remove(i)
  frame_index = frame_index[remain]

  final = []
  for i in range(len(frame_index)):
    final.append(frames[frame_index[i]])

  ssim_dict = {}
  for i in range(len(final)):
    ssim_dict[i] = []
    for j in range(i+1,len(final)):
      ssim_dict[i].append(ssim(final[i],final[j],multichannel=True))
  ssim_list = []
  for sublist in ssim_dict.values():
    for item in sublist:
      ssim_list.append(item)
  threshold_ssim = np.percentile(ssim_list,80)

  delete_list_ssim = set()
  for i in range(len(ssim_dict.keys())):
    for j in range(len(ssim_dict[i])):
      if ssim_dict[i][j]>=threshold_ssim:
        delete_list_ssim.add(i+1+j)
  remain = list(range(len(final)))
  for i in delete_list_ssim:
    remain.remove(i)

  key_frames_remain = []
  for i in range(len(remain)):
    key_frames_remain.append(final[remain[i]])
  
  key_frames_remain2 = list(np.array(key_frames_remain).copy())

  collage(key_frames_remain)

  for i in range(len(key_frames_remain2)):
    key_frames_remain2[i] = (transform.resize(key_frames_remain2[i], 
                  (frames[0].shape))*255).astype('uint8')
  gif_frame_making(key_frames_remain2, 21)

  



  

  

In [12]:
%%time
ROOT1 = "/content/gdrive/My Drive/FIT5221_ass3_30399262_27462870/test_frame_video_family/"
generateCoolSummary(ROOT1)

Output hidden; open in https://colab.research.google.com to view.